In [2]:
#pip install -U scipy
#pip install -U shapely
#pip install -U plotly
#pip install -U dash
#pip install -U networkx
#pip install -U scikit-learn
#pip install -U kaleido

In [3]:
2+2

4

# 🚀 Experiment Execution Flow

In [ ]:
import sys
from pathlib import Path

base_dir = Path.cwd().parent
src_path = base_dir / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

    
from project_setup import (
    setup_paths,
    load_experiments_dict,
    import_and_reload_modules
)

# Set paths and load experiments
base_dir, src_path = setup_paths()
experiments_definition = load_experiments_dict("experiments_config.json")

# Import project classes
WorldModel, DisplayManager, MatrixSimilarity, NetworkManager = import_and_reload_modules()

# Plotly settings
import plotly.io as pio
#pio.renderers.default = "iframe_connected"
pio.renderers.default = "vscode"
# Experiment configuration
experiment_id = "ECSS18"
experiment_display_scale = {
    "ECIR19": 0.28,
    "ECSS18": 0.15,
    "ICCSS17": 0.3,
    "WS16": 0.15
}

# Initialize world & displayer
world = WorldModel(experiment_id, experiments_definition, base_dir)
world.initialize()




✅ Loaded experiment definition from experiments_config.json
✅ Loaded contact data file: tij_with_readers_ECSS18.dat
✅ Converted and renamed 'readers' column to 'signature'.
✅ Loaded periods file: periodes_ECSS18.dat
🔄 Synchronizing periods for ECSS18 (offset: 1h)...
✅ Loaded plan 'Main' (4678x3648) with 8 readers
✅ Computed 105 signatures.


In [5]:

Displayer = DisplayManager(
    world,
    width=720,
    height=720,
    font_size=14,
    title_size=22,
    axis_title_size=16,
    tick_size=12
)


✅ DisplayManager initialized with Plotter, Matrixer, and Networker.
✅ Mapper initialized with 1 plans.


## 🔄 Period Detection & Adjustment Pipeline

In [6]:
# 🔍 Detect Transitions (Activity-based)
debug_data = world.period_manager.detect_transitions(
    threshold_pos=0.1, threshold_neg=-0.05, smooth_sigma=3, freq="1min", debug=True
)

🔍 Detecting transitions for ECSS18...
✅ Detected 33 transitions.
🛠️ Adjusting periods with transitions for ECSS18...
✅ Final periods updated: 30 entries.


if debug_data:
    
    Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.plotter.plot_transition_debug_activity,
    plot_kwargs={
        "activity_series": debug_data["activity_series"],
        "smoothed": debug_data["smoothed"],
        "norm_derivative": debug_data["norm_derivative"],
        "transition_df": debug_data["transition_df"],
        "experiment_id": world.experiment_id,
        "smooth_sigma": debug_data["smooth_sigma"]
    },
    output_basename="transition_debug",
    styler=Displayer.styler
)


Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.plotter.plot_period_shading_stages,
    plot_kwargs={
        "df_initial": world.period_manager.periods_df,
        "df_transition": world.period_manager.transition_df,
        "df_final": world.period_manager.final_period_df,
    },
    output_basename="period_segmentation",
    styler=None
)

In [7]:
simil = MatrixSimilarity()
activity_signature = simil.compute_activity_matrix(
    world.period_manager.final_period_df, world.signatures
)

cosine_signature, signature_group = simil.compute_cosine_similarity(
    activity_input=activity_signature,
    reordered=True,
    num_groups=3,
    return_groups=True,
)


cosine_signature, signature_group = simil.compute_combined_similarity(
    activity_input=activity_signature,
    reordered=True,
    alpha=1,
    num_groups=4,
    return_groups=True,
)


world.define_areas_by_group(signature_group)



def activity_signature_plotter(matrix, title, xlabel="Period", ylabel="Signature", **kwargs):
    return Displayer.matrixer.plot_activity_matrix(
        matrix=matrix,
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        **kwargs
    )

# Then, run the Dash export app using this wrapper
Displayer.run_plotly_dash_export_app(
    plot_func=activity_signature_plotter,
    plot_kwargs={
        "matrix": activity_signature, 
        "title": f"Activity Matrix — {world.experiment_id}",
        "annot": False,
        "linewidth": 0.5,
        "linecolor": "gray",
        "cmap": "Viridis",
    },
    output_basename="activity_matrix",
    styler=Displayer.styler
)

def cosine_similarity_plotter(matrix, title, xlabel="Signature", ylabel="Signature", **kwargs):
    return Displayer.matrixer.plot_cosine_similarity_matrix(
        cos_sim_df=matrix,
        title=title,
        **kwargs
    )
Displayer.run_plotly_dash_export_app(
    plot_func=cosine_similarity_plotter,
    plot_kwargs={
        "matrix": cosine_signature,  
        "title": f"Cosine Similarity Matrix — {world.experiment_id}",
        "annot": False,
        "linewidth": 0.5,
        "linecolor": "gray",
        "cmap": "Viridis",
    },
    output_basename="cosine_similarity_matrix",
    styler=Displayer.styler
)


Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.plotter.plot_normalized_activity,
    plot_kwargs={
        "entities": world.areas,
        "experiment_id": world.experiment_id,
        "mode_label": "Areas",
        "title_suffix": "",
        "show_total": True,
        "sort_by_activity": True,
        "df_period": world.period_manager.final_period_df,
    },
    output_basename="normalized_activity",
    styler=Displayer.styler
)

In [8]:
world.assign_agents_to_areas_over_time(freq="20min")
world.compute_active_agent_to_area()

✅ Assigned 164 agents with position over time.


In [9]:
Networker = NetworkManager()
Networker.build_area_transition_graph(world)
Networker.build_temporal_area_transition_graphs(world)

In [10]:
Networker.build_temporal_agent_graphs_by_area(world,'10min')

✅ Built fixed-window temporal contact graphs in all Area objects.


# === 📈 Plotter Methods ===
Displayer.plotter.plot_normalized_activity(
    entities=world.areas,
    experiment_id=world.experiment_id,
    mode_label="areas",
    title_suffix="",
    show_total=True,
    sort_by_activity=True,
    df_period=world.period_manager.final_period_df,
)

In [11]:
Networker.export_node_edge_timeseries_per_area(world)

[i] Cleared 2 old .csv file(s) in 'densification_data'.
[✓] Saved: densification_data/nm_main_1.csv
[✓] Saved: densification_data/nm_main_2.csv
[!] Skipped Main_3 (no valid (N,M) data)
[!] Skipped Main_4 (no valid (N,M) data)


Displayer.networker.plot_node_edge_distribution_per_area(world)



Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.networker.plot_area_transition_sankey,
    plot_kwargs={
        "G": Networker.area_transition_graph,
        "world": world,
    },
    output_basename="sankey",
    styler=Displayer.styler
)

In [12]:
world.compute_agents_entropies()

✅ Computed entropy for 164 agents.


Displayer.plotter.plot_agent_entropy_distribution(world.agents)




Displayer.run_plotly_dash_export_app(
    plot_func=lambda **kwargs: Displayer.networker.network_display(**kwargs),
    plot_kwargs={
        "graph": Networker.area_transition_graph,
        "experiment_id": world.experiment_id,
        "title": "Interactive Area Transition Network"
    },
    output_basename="area_network_dash"
)


def normalized_area_plot(**kwargs):
    return Displayer.plotter.plot_normalized_activity(
        entities=world.areas,
        experiment_id=world.experiment_id,
        mode_label="areas",
        title_suffix="",
        show_total=True,
        sort_by_activity=True,
        df_period=world.period_manager.final_period_df,
        **kwargs
    )
Displayer.run_plotly_dash_export_app(
    plot_func=normalized_area_plot,
    plot_kwargs={},  # or add any overrides here
    output_basename="normalized_activity_areas",
    styler=Displayer.styler
)


Displayer.plotter.plot_signature_distributions()

Displayer.plotter.periods_df = world.period_manager.final_period_df
Displayer.plotter.report_world_model()

Displayer.plotter.report_periods()

Displayer.matrixer.plot_activity_matrix(
        matrix=activity_signature,
        title=f"Activity Matrix — Experiment: {experiment_id}",
        xlabel="Period",
        ylabel="Entity",
        cmap="Viridis",
        linewidth=0.5,
        linecolor="gray",
        annot=False,
        fmt=".0f",
    )



Displayer.matrixer.plot_cosine_similarity_matrix(
    cos_sim_df=cosine_signature,
    title=f"Cosine Similarity Between Signatures — Experiment: {experiment_id}",
    cmap="Viridis",
    linewidth=0.5,
    linecolor="gray",
    annot=False,
    fmt=".2f",
)

In [15]:
# === 🗺️ Mapper Methods ===
Displayer.mapper.display(scale=experiment_display_scale[experiment_id])


In [16]:
Displayer.mapper.plot_animated_signature_activity(
    grid_size=(20, 20),
    sigma=0.4,
    scale=experiment_display_scale[experiment_id],
    heatmap_opacity=0.8,
    time_bin="4h"
)

Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.mapper.display, 
    plot_kwargs={
        "scale": experiment_display_scale[experiment_id],
        "activity_threshold" : 0,
        "show_outline" : False
    },
    output_basename="map_dis"
)


Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.mapper.display_with_gaussian_smoothing, 
    plot_kwargs={
        "scale": experiment_display_scale[experiment_id],
        "sigma":20,
    },
    output_basename="map_dis_gaus"
)

Displayer.run_plotly_dash_export_app(
    plot_func=Displayer.mapper.plot_animated_signature_activity, 
    plot_kwargs={
        "scale": experiment_display_scale[experiment_id],
        "grid_size" : (20, 20),
        "sigma" : 1,
        "heatmap_opacity" : 0.5,
        "time_bin" : "2h"
        
    },
    output_basename="map_animated_"
)
